In [1]:
# DONE:
# add SVR model
# get prediction as combination of best models: games * per_game

# TODO:
# add feats cat (label, OH)
# staking of the best models

### Functions

In [2]:
def get_datasets(df, feat_target, feat_norm, norm_shift, flg_ma):
    season_min = 2023
    # calculate mean and std
    eps = 1e-3
    feats_static = [
        'static_height_cm',
        'static_weight_kg',
        'static_bmi_calc',
    ]
    if feat_norm != None:
        df_ag = df.groupby(feat_norm)[feat_target].agg([np.mean, np.std]).fillna(0)
        df_ag.columns = ['m','s']
        df_ag.reset_index(inplace = True)
        # create lag -1 to prevent look into the future
        df_ag['season'] = df_ag['season'] + norm_shift
        df_ag['s'] = df_ag['s'] + eps
        df = df.merge(df_ag, how = 'left')
        
        # normalize static features
        for f in feats_static:
            df[f] = (df[f] - df[f].mean()) / df[f].std()
    else:
        df['m'] = 0
        df['s']  = 1 + eps

    # normalization of target
    df['target'] = (df[feat_target] - df['m']) / df['s'] 

    # create lag features of the target
    max_lag = season_min - 2016
    for i in range(1,max_lag):
        df[f'lag{i}'] = df.groupby('player_id')['target'].shift(i)
    feats_lag = [f'lag{i}' for i in range(1, max_lag)]

    # feats for training
    feats_used = feats_static + feats_lag

    # moving averages features
    if flg_ma:
        for i in range(1,max_lag):
            df[f'ma{i}'] = df.loc[:,'lag1':f'lag{i}'].mean(axis = 1)
        feats_ma = [f'ma{i}' for i in range(1, max_lag)]
        feats_used = feats_used + feats_ma

    # nulls filling (for sklearn models)
    for f in feats_lag:
        df[f] = df[f].fillna(-1)

    # filtering
    filt = df.season >= season_min
    feats_info = ['player_id','display_name','season','position','fantasy_points_ppr','games','fantasy_points_per_game']
    return df[filt][feats_used], df[filt]['target'], df[filt]['m'], df[filt]['s'], df[filt][feats_info]

In [3]:
# utility function to get kfold avg and std scores
def cv_scores(clf, kfold):
    pred = ms.cross_val_predict(clf, x, y, cv = kfold)
    pred = (pred * s + m).values
    list_res = list()
    for cv in kfold.split(real):
        ind = list(cv[1])
        list_res.append(metrics.mean_absolute_error(pred[ind], real[ind]))
    return np.mean(list_res), np.std(list_res)


def model_searching(kfold1, kfold2):
    dict_out = dict()
    dict_params = dict()
    
    ###
    modelname = 'Decision tree'
    from sklearn.tree import DecisionTreeRegressor
    clf = DecisionTreeRegressor()
    params = {
        'max_depth': list(range(2,10)),
    }
    grid = ms.GridSearchCV(clf, params, cv = kfold1, scoring = 'neg_mean_absolute_error')
    grid.fit(x, y)
    clf = DecisionTreeRegressor(**grid.best_params_)
    dict_out[modelname] = cv_scores(clf = clf, kfold = kfold2)
    dict_params[modelname] = str(grid.best_params_)
    
    ###
    modelname = 'Xgboost'
    import xgboost as xgb
    clf = xgb.XGBRegressor()
    params = {
        'learning_rate': [0.05, 0.1],
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 6, 9],
        'gamma': [0, 1],
        'subsample': [0.7, 1],
        'colsample_bytree': [0.7, 1]
    }
    grid = ms.GridSearchCV(clf, params, cv = kfold1, scoring = 'neg_mean_absolute_error')
    grid.fit(x, y)
    clf = xgb.XGBRegressor(**grid.best_params_)
    dict_out[modelname] = cv_scores(clf = clf, kfold = kfold2)
    dict_params[modelname] = str(grid.best_params_)
    
    ###
    modelname = 'Random forest'
    from sklearn.ensemble import RandomForestRegressor
    clf = RandomForestRegressor(n_estimators=100)
    params = {
        'max_depth': [3, 6, None],
    }
    grid = ms.GridSearchCV(clf, params, cv = kfold1, scoring = 'neg_mean_absolute_error')
    grid.fit(x, y)
    clf = RandomForestRegressor(**grid.best_params_)
    dict_out[modelname] = cv_scores(clf = clf, kfold = kfold2)
    dict_params[modelname] = str(grid.best_params_)
    
    ###
    modelname = 'KNN'
    from sklearn.neighbors import KNeighborsRegressor
    clf = KNeighborsRegressor(n_neighbors=2)
    params = {
        'n_neighbors': [2, 5, 10, 25, 50],
    }
    grid = ms.GridSearchCV(clf, params, cv = kfold1, scoring = 'neg_mean_absolute_error')
    grid.fit(x, y)
    clf = KNeighborsRegressor(**grid.best_params_)
    dict_out[modelname] = cv_scores(clf = clf, kfold = kfold2)
    dict_params[modelname] = str(grid.best_params_)
    
    ###
    modelname = 'Linear model'
    from sklearn.linear_model import ElasticNet
    clf = ElasticNet(alpha = 0.005)
    params = {
        'alpha': [1, 0.5, 0.1, 0.05, 0.01, 0.005, 0.001],
        'l1_ratio': [10, 2, 1, 0.5, 0.1, 0.01],
    }
    grid = ms.GridSearchCV(clf, params, cv = kfold1, scoring = 'neg_mean_absolute_error')
    grid.fit(x, y)
    clf = ElasticNet(**grid.best_params_)
    dict_out[modelname] = cv_scores(clf = clf, kfold = kfold2)
    dict_params[modelname] = str(grid.best_params_)

    ###
    modelname = 'Support Vectors'
    from sklearn.svm import SVR
    clf = SVR(C=1.0, epsilon=0.2)
    params = {
        'C': [10, 5, 1, 0.5, 0.1],
        'epsilon': [1, 0.1, 0.01],
        'kernel': ['linear','poly','rbf','sigmoid']
    }
    grid = ms.GridSearchCV(clf, params, cv = kfold1, scoring = 'neg_mean_absolute_error')
    grid.fit(x, y)
    clf = SVR(**grid.best_params_)
    dict_out[modelname] = cv_scores(clf = clf, kfold = kfold2)
    dict_params[modelname] = str(grid.best_params_)
    
    ###
    modelname = 'Neural network (MLP)'
    from sklearn.neural_network import MLPRegressor
    clf = MLPRegressor()
    params = {
        'activation' : ['identity', 'logistic', 'tanh', 'relu'],
        'hidden_layer_sizes': [(2,), (4,), (8,), (16,), (2,2), (4,2), (8,2) ],
    }
    grid = ms.GridSearchCV(clf, params, cv = kfold1, scoring = 'neg_mean_absolute_error')
    grid.fit(x, y)
    clf = MLPRegressor(**grid.best_params_)
    dict_out[modelname] = cv_scores(clf = clf, kfold = kfold2)
    dict_params[modelname] = str(grid.best_params_)

    ###
    # moving averages
    if flg_ma:
        for i in range(1,6):
            pred = x[f'ma{i}']
            pred = (pred * s + m).values
            list_res = list()
            for cv in kfold2.split(real):
                ind = list(cv[1])
                list_res.append(metrics.mean_absolute_error(pred[ind], real[ind]))
            dict_out[f'MA{i}'] = np.mean(list_res), np.std(list_res)
        
    return dict_out, dict_params

### Training

In [4]:
%%time
#####################
# TRAINING
#####################

# The targets we want to predict (comparing "total" vs. "games" * "per game")
feats_target = [
    'games',
    'fantasy_points_per_game',
    'fantasy_points_ppr',
]
# Normalization features
feats_norm = [
    None,                        # without normalization
    ['season'],                  # only season
    ['season','position'],       # by the position + season
    ['season','player_cluster'], # by the player cluster + season
    ['season','team_cluster'],   # by the team cluster + season
]
norm_shift = 1 # number of seasons to shift for normalization, 0 - for the new 2024 predictions, 1 - for training 

########## 
# read main preprocessed dataset
df = pd.read_pickle('data/df_model.pkl')

from sklearn import model_selection as ms
from sklearn import metrics
kfold1 = ms.KFold(n_splits = 5, shuffle = True, random_state = 2017) # k folds for models parameters searching
kfold2 = ms.KFold(n_splits = 5, shuffle = True, random_state = 2022) # k folds for best models evaluation

# main loop
df_out_mean = pd.DataFrame()
df_out_std  = pd.DataFrame()
df_param = pd.DataFrame()
for feat_target in feats_target:
    for feat_norm in feats_norm:
        for flg_ma in [True, False]:
            try:
                print(feat_target, feat_norm, flg_ma)
                # 1) create datasets
                x, y, m, s, df_out = get_datasets(df = df[df.season_len > 1], 
                                          feat_target = feat_target, 
                                          feat_norm = feat_norm, 
                                          norm_shift = norm_shift, 
                                          flg_ma = flg_ma)
                real = (y * s + m).values

                # 2) searching the best models parameters on kfold1 and get their scores on kfold2 
                res1, param1 = model_searching(kfold1, kfold2)

                df_out1 = pd.DataFrame(res1)
                df_out1['target'] = feat_target
                df_out1['norm'] = str(feat_norm)
                df_out1['ma'] = flg_ma
                df_out_mean = pd.concat([df_out_mean, df_out1[:1]])
                df_out_std = pd.concat([df_out_std, df_out1[1:]])
                
                df_out2 = pd.DataFrame([[k,v] for k,v in param1.items()]).set_index(0).T
                df_out2['target'] = feat_target
                df_out2['norm'] = str(feat_norm)
                df_out2['ma'] = flg_ma
                df_param = pd.concat([df_param, df_out2])
                
            except Exception as e:
                print(e)

games None True
games None False
games ['season'] True
games ['season'] False
games ['season', 'position'] True
games ['season', 'position'] False
games ['season', 'player_cluster'] True
games ['season', 'player_cluster'] False
games ['season', 'team_cluster'] True
games ['season', 'team_cluster'] False
fantasy_points_per_game None True
fantasy_points_per_game None False
fantasy_points_per_game ['season'] True
fantasy_points_per_game ['season'] False
fantasy_points_per_game ['season', 'position'] True
fantasy_points_per_game ['season', 'position'] False
fantasy_points_per_game ['season', 'player_cluster'] True
fantasy_points_per_game ['season', 'player_cluster'] False
fantasy_points_per_game ['season', 'team_cluster'] True
fantasy_points_per_game ['season', 'team_cluster'] False
fantasy_points_ppr None True
fantasy_points_ppr None False
fantasy_points_ppr ['season'] True
fantasy_points_ppr ['season'] False
fantasy_points_ppr ['season', 'position'] True
fantasy_points_ppr ['season', 'po

### Scoring tables

In [6]:
target = 'fantasy_points_ppr'
print('Average MAE scores')
filt = df_out_mean.target == target
df_out1 = df_out_mean[filt]
del df_out1['target']
display(df_out1.set_index(['norm','ma']).ffill().T.style.background_gradient(cmap='RdYlGn_r').set_precision(1))

print('STD of MAE scores')
filt = df_out_std.target == target
df_out1 = df_out_std[filt]
del df_out1['target']
display(df_out1.set_index(['norm','ma']).ffill().T.style.background_gradient(cmap='RdYlGn_r').set_precision(1))

Average MAE scores


STD of MAE scores


In [7]:
target = 'games'
print('Average MAE scores')
filt = df_out_mean.target == target
df_out1 = df_out_mean[filt]
del df_out1['target']
display(df_out1.set_index(['norm','ma']).ffill().T.style.background_gradient(cmap='RdYlGn_r').set_precision(1))

print('STD of MAE scores')
filt = df_out_std.target == target
df_out1 = df_out_std[filt]
del df_out1['target']
display(df_out1.set_index(['norm','ma']).ffill().T.style.background_gradient(cmap='RdYlGn_r').set_precision(1))

Average MAE scores


STD of MAE scores


In [8]:
target = 'fantasy_points_per_game'
print('Average MAE scores')
filt = df_out_mean.target == target
df_out1 = df_out_mean[filt]
del df_out1['target']
display(df_out1.set_index(['norm','ma']).ffill().T.style.background_gradient(cmap='RdYlGn_r').set_precision(1))

print('STD of MAE scores')
filt = df_out_std.target == target
df_out1 = df_out_std[filt]
del df_out1['target']
display(df_out1.set_index(['norm','ma']).ffill().T.style.background_gradient(cmap='RdYlGn_r').set_precision(1))

Average MAE scores


STD of MAE scores


### CV predictions of best models

In [53]:
from sklearn import model_selection as ms
from sklearn import metrics
from sklearn.svm import SVR

kfold3 = ms.KFold(n_splits = 5, shuffle = True, random_state = 2016)

# 0) choose best model
target = 'fantasy_points_ppr'
flg_ma = False
feat_norm = None

f1 = df_param.target == target
f2 = df_param.ma == flg_ma
f3 = df_param.norm == str(feat_norm)
params = eval(df_param[f1&f2&f3]['Support Vectors'].values[0])
print(params)
clf = SVR(**params)

# 1) create dataset for training
x, y, m, s, df_out = get_datasets(df = df[df.season_len > 1], 
                          feat_target = target, 
                          feat_norm = feat_norm, 
                          norm_shift = 1,
                          flg_ma = flg_ma)

# 2) cross val predictions
pred = ms.cross_val_predict(clf, x, y, cv = kfold3)
pred = pred * s + m
pred0 = pred

metrics.mean_absolute_error(y, pred)

{'C': 10, 'epsilon': 1, 'kernel': 'linear'}


40.96288945482981

In [54]:
y1 = y

In [55]:
from sklearn import model_selection as ms
from sklearn import metrics
from sklearn.svm import SVR

kfold3 = ms.KFold(n_splits = 5, shuffle = True, random_state = 2016)

# 0) choose best model
target = 'games'
flg_ma = False
feat_norm = None

f1 = df_param.target == target
f2 = df_param.ma == flg_ma
f3 = df_param.norm == str(feat_norm)
params = eval(df_param[f1&f2&f3]['Support Vectors'].values[0])
print(params)
clf = SVR(**params)

# 1) create dataset for training
x, y, m, s, df_out = get_datasets(df = df[df.season_len > 1], 
                          feat_target = target, 
                          feat_norm = feat_norm, 
                          norm_shift = 1,
                          flg_ma = flg_ma)

# 2) cross val predictions
pred = ms.cross_val_predict(clf, x, y, cv = kfold3)
pred = pred * s + m

pred1 = pred

{'C': 0.1, 'epsilon': 0.1, 'kernel': 'linear'}


In [56]:
from sklearn import model_selection as ms
from sklearn import metrics
from sklearn.svm import SVR

kfold3 = ms.KFold(n_splits = 5, shuffle = True, random_state = 2016)

# 0) choose best model
target = 'fantasy_points_per_game'
flg_ma = False
feat_norm = None

f1 = df_param.target == target
f2 = df_param.ma == flg_ma
f3 = df_param.norm == str(feat_norm)
params = eval(df_param[f1&f2&f3]['Support Vectors'].values[0])
print(params)
clf = SVR(**params)

# 1) create dataset for training
x, y, m, s, df_out = get_datasets(df = df[df.season_len > 1], 
                          feat_target = target, 
                          feat_norm = feat_norm, 
                          norm_shift = 1,
                          flg_ma = flg_ma)

# 2) cross val predictions
pred = ms.cross_val_predict(clf, x, y, cv = kfold3)
pred = pred * s + m

pred2 = pred

{'C': 0.1, 'epsilon': 0.01, 'kernel': 'linear'}


In [57]:
# combination of predictions
pred = pred1*pred2

metrics.mean_absolute_error(y1, pred)

43.41129650051318

In [58]:
# best prediction
metrics.mean_absolute_error(y1, pred0)

40.96288945482981

### Make predictions

In [63]:
#####################
# PREDICTING
#####################

# 0) choose best model
from sklearn.svm import SVR

target = 'fantasy_points_ppr'
flg_ma = False
feat_norm = None

f1 = df_param.target == target
f2 = df_param.ma == flg_ma
f3 = df_param.norm == str(feat_norm)
params = eval(df_param[f1&f2&f3]['Support Vectors'].values[0])
print(params)
clf = SVR(**params)

# 1) create datasets
# for training
x, y, m, s, df_out = get_datasets(df = df[df.season_len > 1], 
                          feat_target = target, 
                          feat_norm = feat_norm, 
                          norm_shift = 1, 
                          flg_ma = flg_ma)
print(x.shape)
# for predictions
x1, y1, m1, s1, df_out1 = get_datasets(df = df,   # change here
                          feat_target = target, 
                          feat_norm = feat_norm, 
                          norm_shift = 0,         # change here
                          flg_ma = flg_ma)
print(x1.shape)

# 2) train model with the best params
clf.fit(x, y)

# 3) make predictions
pred = clf.predict(x1)
df_out1['pred'] = (pred * s1 + m1).values
df_out1 = df_out1[['display_name','pred']]


# 4) add Fullback position predictions and save to file
df_out2 = pd.read_csv('data/output_add1.csv')
df_out1.columns = df_out2.columns

df_out = pd.concat([df_out1, df_out2])
df_out.to_csv('data/output_nfl.csv', index = False)
print(df_out.shape)
df_out.sort_values('Projected fantasy points scored PPR for 2023', ascending = False)[:10]

{'C': 10, 'epsilon': 1, 'kernel': 'linear'}
(444, 9)
(560, 9)
(572, 2)


,Player Name,Projected fantasy points scored PPR for 2023
1066,Patrick Mahomes,361.51
1445,Josh Allen,355.13
1120,Cooper Kupp,346.31
972,Deshaun Watson,320.35
1856,Justin Herbert,286.89
1015,Austin Ekeler,283.73
7,Tom Brady,271.13
1776,Jonathan Taylor,268.84
388,Davante Adams,259.69
1108,Joe Mixon,254.03
